## Word embedding using word2vec and similarity metrics (softcosine)

In [1]:
import json
import logging
from re import sub
from multiprocessing import cpu_count

import numpy as np
import pandas as pd 

import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import MatrixSimilarity, SoftCosineSimilarity

import nltk
from nltk.corpus import wordnet as wn

from gensim.models import Word2Vec

# pip install pymed

In [2]:
# Support functions for pre-processing and calculation

# Import and download stopwords from NLTK.
# nltk.download('stopwords')  # Download stopwords list.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb

# Remove stop words and punctuations with nltk
stopwords = set(nltk.corpus.stopwords.words("english"))
punc = ['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%"]
stopwords1 = stopwords.union(punc)

def preprocess(doc):
    #return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords1]


In [3]:
def extract_ids(dicdata):
    ids = []
    for i in dict_data:
        mea_id = i['id']
        ids.append(mea_id)
    return ids

def extract_meanings(dicdata):
    meaning_list = []
    for i in dict_data:
        meaning = i['meaning'] 
        meaning_list.append(meaning)
    return meaning_list

def extract_isteacher(dicdata):
    isteacher = []
    for i in dict_data:
        is_tea = i['isTeacher']
        isteacher.append(is_tea)
    return isteacher

In [4]:
def extract_ids(dicdata):
    ids = []
    for i in dict_data:
        mea_id = i['id']
        ids.append(mea_id)
    return ids

def extract_meanings(dicdata):
    meaning_list = []
    for i in dict_data:
        meaning = i['meaning'] 
        meaning_list.append(meaning)
    return meaning_list

def extract_pos_meanings(dicdata):
    pos_meaning = []
    for i in dict_data:
        pos = i['fl']
        meaning = i['meaning'] 
        pos_mea = pos + ' '+ meaning # POS + meaning
        pos_meaning.append(pos_mea)
    return pos_meaning

def extract_isteacher(dicdata):
    isteacher = []
    for i in dict_data:
        is_tea = i['isTeacher']
        isteacher.append(is_tea)
    return isteacher

### Measure: Soft cosine Similarity

In [5]:
# Load Data and store
jsonfile= open('data_600.json','r',encoding='utf8')
data= jsonfile.read() 

# Parse and Extract word, paragraph, pos and meanings
obj = json.loads(data)

data = obj['data'] #[:10]
print("Total words: ",len(data)) # Total # of words
count=0
# Store accuracy and MRR in blank lists
acc_score=[] 
mrr_score = []

# Parse data and find best meaning
for each_word_data in data:
    word = each_word_data['word']
    paragraph = each_word_data['paragraph']
    dict_data = each_word_data['dictionaryData']
    for i in dict_data:
        ids = extract_ids(dict_data)
        meanings = extract_meanings(dict_data)
        isteacher = extract_isteacher(dict_data)
        
        
    # Preprocess the documents, including the query string(paragraph)
    corpus = []
    for i in meanings:
        corpus.append(preprocess(i))
    query = preprocess(paragraph)



    # Build the model
    # The word embedding model is a large file, so loading is quite a long-running task.

    # Build the term dictionary, TF-idf model
    # The search query must be in the dictionary as well, in case the terms do not overlap with the documents (we still want similarity)
    
    dictionary = Dictionary(corpus+[query])
    tfidf = TfidfModel(dictionary=dictionary)

    # Create the term similarity matrix. 
    # The nonzero_limit enforces sparsity by limiting the number of non-zero terms in each column. 
    # removing the default value of 100
    
    # Default min_count in gensim's Word2Vec is set to 5. If there is no word in your vocab with frequency greater than 4,
    #your vocab will be empty and hence the error. try 1.
    w2v_model = Word2Vec(corpus+[query], workers=cpu_count(), min_count=1, size=300, alpha=0.025, seed=1)
    similarity_index = WordEmbeddingSimilarityIndex(w2v_model.wv)
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf, nonzero_limit=None)  # nonzero_limit=None)


    # Compute Soft Cosine Measure between the query and the documents.
    query_tf = tfidf[dictionary.doc2bow(query)]

    index = SoftCosineSimilarity(
                tfidf[[dictionary.doc2bow(document) for document in corpus]],
                similarity_matrix)

    doc_similarity_scores = (index[query_tf]).round(3)

    # Output the document similarity results
    #sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
    #print(sorted_indexes)

    zipped = list(zip(ids, meanings, doc_similarity_scores, isteacher))
    features = sorted(zipped, key = lambda x: x[2], reverse = True)

    print("\n\nLookup word: ",word)
    ranked_meanings = pd.DataFrame(features, columns = ['id', 'meaning','score','isTeacher'])
    print(ranked_meanings)
    count +=1
    print(count)
    
    # Find accuracy
    if ranked_meanings.iloc[0]['isTeacher'] ==1:
        acc_score.append(1)
    else:
        acc_score.append(0)

    
    # Find MRR score
    istea_index = ranked_meanings[ranked_meanings['isTeacher']==1].index[0]+1
    pred_rank =1/istea_index
    mrr_score.append(pred_rank)

    
# Return Accuracy and MRR    
#print(acc_score)
avg = sum(acc_score)/len(acc_score)
print("The accuracy score is ", round(avg,4))    

mrr = np.sum(mrr_score)/ len(mrr_score)
print("The MRR is ", round(mrr,4)) 

Total words:  600


Lookup word:  circumvent 
     id                                            meaning  score  isTeacher
0  3230  to avoid being stopped by (something, such as ...  0.042          1
1


Lookup word:  addict
     id                                            meaning  score  isTeacher
0  5884  one exhibiting a compulsive, chronic, physiolo...  0.053        1.0
1  3401  a person who is not able to stop taking drugs ...  0.031        NaN
2  5885                    to cause addiction in (someone)  0.027        NaN
2


Lookup word:  frustrated
     id                                            meaning  score  isTeacher
0  5886  feeling, showing, or characterized by frustrat...  0.024        1.0
1  5887                   to balk or defeat in an endeavor  0.020        NaN
3


Lookup word:  manslaughter
     id                                            meaning  score  isTeacher
0  2554  the crime of killing a person without intendin...  0.027          1
4


Lookup word:  blud

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))




Lookup word:  copious
     id                         meaning  score  isTeacher
0  2558  very large in amount or number  0.026        1.0
1  5933   yielding something abundantly  0.021        NaN
21


Lookup word:  zeal
     id                                            meaning  score  isTeacher
0  5935                                   great enthusiasm  0.057        NaN
1  3319  a strong feeling of interest and enthusiasm th...  0.047        1.0
2  5934  eagerness and ardent interest in pursuit of so...  0.030        NaN
3  3320                     of or relating to missionaries  0.013        NaN
22


Lookup word:  convictions
     id                                            meaning  score  isTeacher
0  5937       the courage to do what one believes is right  0.062        1.0
1  5936  the act or process of finding a person guilty ...  0.040        NaN
2  2575  the act of proving that a person is guilty of ...  0.033        NaN
3  2576  the ability to do something that you know is 

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))




Lookup word:  hysteria
     id                                            meaning  score  isTeacher
0  5952  a psychoneurosis in which bodily symptoms (suc...  0.044        NaN
1  5951  a psychoneurosis marked by emotional excitabil...  0.022        NaN
2  3324  a state in which your emotions (such as fear) ...  0.015        1.0
29


Lookup word:  misapprehension
     id                                            meaning  score  isTeacher
0  5954  to continue to believe something that is not true  0.016        NaN
1  3478  an incorrect understanding of something : a wr...  0.012        1.0
2  3479                                         to do work  0.009        NaN
3  5953               to apprehend wrongly : misunderstand  0.008        NaN
30


Lookup word:  pious
     id                                            meaning  score  isTeacher
0  5955  marked by or showing reverence for deity and d...  0.015        NaN
1  2633  a desire for something to happen or be done : ...  0.009   

...



Lookup word:  docket
     id                                            meaning  score  isTeacher
0  1558  a list of the legal cases that will be tried i...  0.018          1
505


Lookup word:  bail
     id                                            meaning  score  isTeacher
0  4568   to spring or cause to spring into the air : leap  0.087        NaN
1  4563  to entrust (personal property) to another for ...  0.042        NaN
2  4569  the giving of oneself or something into the po...  0.036        NaN
3  4562  security that guarantees the appearance of a p...  0.029        1.0
4  4561  to remove (water) from a boat by dipping and t...  0.028        NaN
5  4566  a large bundle of goods; especially : one clos...  0.022        NaN
6  4567  to jump with a parachute from an airplane in f...  0.019        NaN
7  4560       a container used to remove water from a boat  0.018        NaN
8  4564                             a supporting half hoop  0.017        NaN
9  4565                     



Lookup word:  Overt
     id                  meaning  score  isTeacher
0  4761  open to view : manifest  0.027          1
532


Lookup word:  assault
     id                                            meaning  score  isTeacher
0  4612  any of various automatic or semiautomatic rifl...  0.040        NaN
1  4611                            to make an assault upon  0.022        NaN
2  4610    a violent physical or verbal attack : onslaught  0.020        1.0
533


Lookup word:  bartered
     id                                            meaning  score  isTeacher
0  6891  to trade by exchanging one commodity for anoth...  0.031          1
534


Lookup word:  homicides
     id                                            meaning  score  isTeacher
0  6868  the deliberate and unlawful killing of one per...  0.038          1
1  4756                         a person who kills another  0.014          0
535


Lookup word:  haphazard
     id                                            meaning  score 

C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: divide by zero encountered in true_divide
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))
C:\Users\Swati\AppData\Local\Continuum\anaconda3\envs\nlpenv\lib\site-packages\gensim\similarities\termsim.py:358: RuntimeWarning: invalid value encountered in multiply
  Y = np.multiply(Y, 1 / np.sqrt(Y_norm))




Lookup word:  equivocating
     id                                            meaning  score  isTeacher
0  4842  to use unclear language especially with intent...  0.019          1
560


Lookup word:  anguish
     id                                   meaning  score  isTeacher
0  4040         extreme suffering, grief, or pain  0.011        1.0
1  4843  extreme pain or distress of body or mind  0.004        NaN
561


Lookup word:  compelled
     id                                      meaning  score  isTeacher
0  6926  to drive or urge forcefully or irresistibly  0.010        NaN
1  6927           to feel required (to do something)  0.008        1.0
562


Lookup word:  vanished
     id                                 meaning  score  isTeacher
0  6928  to pass quickly from sight : disappear  0.018          1
563


Lookup word:  Delta
     id                                            meaning  score  isTeacher
0  6046  low-lying region in NW Miss., extending eastwa...  0.032        1.0
1



Lookup word:  alternative
      id                                            meaning  score  isTeacher
0   3481  something that can be chosen instead of someth...  0.077        1.0
1   6158  a U.S. federal income tax that was originally ...  0.077        NaN
2   6161  alternative music that blends elements of conv...  0.067        NaN
3   6153  a proposition or situation offering a choice b...  0.055        NaN
4   6159  music that is produced by performers who are o...  0.044        NaN
5   6160  pop music that has broad appeal but that is pr...  0.042        NaN
6   6156  a fuel for internal combustion engines that is...  0.041        NaN
7   6157  any of various systems of healing or treating ...  0.038        NaN
8   6155  usable power (such as heat or electricity) tha...  0.023        NaN
9   6154                                        alt-country  0.020        NaN
10  3480                    offering or expressing a choice  0.010        NaN
597


Lookup word:  reprimand
     i